In [281]:
import argparse
import os, sys
# import time
# import tabulate

import torch
import torch.nn.functional as F
# import torchvision
import os
os.sys.path.append("/home/izmailovpavel/Documents/Projects/private_swa_uncertainties/")
from swag import data, models, utils, losses
from swag.posteriors import SWAG
import numpy as np

%matplotlib inline
from sklearn import datasets
import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
# import numpy as np

## Data

In [282]:
# wine
# raw_data = np.loadtxt("/home/izmailovpavel/datasets/UCI/Regression/winequality-red.csv", delimiter=";", skiprows=1)

# concrete
# raw_data = pd.ExcelFile("/home/izmailovpavel/datasets/UCI/Regression/Concrete_Data.xls")

# CCPP
x = np.load("/home/izmailovpavel/datasets/UCI/Regression/CCPP/x.npy")
y = np.load("/home/izmailovpavel/datasets/UCI/Regression/CCPP/y.npy")
raw_data = np.hstack([x, y])


Make a random $0.9$ to $0.1$ train-test split

In [283]:
n_train = int(0.9 * raw_data.shape[0])
np.random.seed(1)
np.random.shuffle(raw_data)
y = raw_data[:, -1]
x = raw_data[:, :-1]
x_tr, y_tr = x[:n_train], y[:n_train]
x_te, y_te = x[n_train:], y[n_train:]

In [284]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
scaler_y = StandardScaler()
# print(x_tr)
x_tr = scaler_x.fit_transform(x_tr)
# print(x_tr)
# print(x_te)
x_te = scaler_x.transform(x_te)
# print(x_te)
y_tr = scaler_y.fit_transform(y_tr[:, None])[:, 0]
y_te = scaler_y.transform(y_te[:, None])[:, 0]

Make pytorch data loader

In [285]:
class DatasetFromNumpy(torch.utils.data.dataset.Dataset):
    def __init__(self, x, y, cuda=False):
        self.x = torch.from_numpy(x).float()
        self.y = torch.from_numpy(y).float()
        if cuda:
            self.x.cuda()
            self.y.cuda()
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return self.y.numel()

In [286]:
trainset = DatasetFromNumpy(x_tr, y_tr)
testset = DatasetFromNumpy(x_te, y_te)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [287]:
for x, y in testloader:
# for x, y in trainloader:
    break

Define the network

In [288]:
class RegressionNetBase(torch.nn.Module):
    def __init__(self, num_inputs, n_hidden=50, p=0.):
        """Using the architecture from Concrete Dropout"""
        super(RegressionNetBase, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Dropout(p),
            torch.nn.Linear(num_inputs, n_hidden),
            torch.nn.ReLU(True),
            torch.nn.Dropout(p),
            torch.nn.Linear(n_hidden, n_hidden),
            torch.nn.ReLU(True),
            torch.nn.Dropout(p),
            torch.nn.Linear(n_hidden, 2)
        )
        
        
    def forward(self, x):
        return self.net(x)
    
class RegressionNet:
    base = RegressionNetBase
    args = list()
    kwargs = {} 

Define the loss

In [289]:
def NLL_(mus, sigmas2, target):
    loss = ((mus - target)**2 / (sigmas2) + torch.log(sigmas2) + torch.log(torch.Tensor([3.1415 * 2]))) / 2
    loss = torch.mean(loss)

    return loss
    

def NLL(model, input, target):
    # standard cross-entropy loss function

    output = model(input)
    n = len(input)
    mus = output[:, 0]
    log_sigmas2 = output[:, 1]
    sigmas2 = torch.exp(log_sigmas2)
    
    
    loss = ((mus - target)**2 / (sigmas2) + log_sigmas2 + torch.log(torch.Tensor([3.1415 * 2]))) / 2
#     print(loss.shape)
    loss = torch.mean(loss)

    return loss, output

SGD Training

In [326]:
model = RegressionNet.base(num_inputs=x_tr.shape[1])

In [327]:
optimizer = torch.optim.SGD(
    model.parameters(),
    #lr=1e-5, # wine
    lr=5e-4, # CCPP
    momentum=0.9,
    weight_decay=5e-2
)

In [328]:
for i in range(400):
    train_res = utils.train_epoch(trainloader, model, NLL, optimizer, cuda=False, regression=True)
    print(train_res['loss'])
    if i == 200:
        utils.adjust_learning_rate(optimizer, lr=5e-5)

1.1787230660034629
0.44345936821169063
0.11460787731422571
0.06440493340913656
0.045386617585987235
0.036021674133429694
0.027325977285887607
0.021586240923040602
0.015675506778965564
0.01069341611189372
0.009793044423208379
0.009363220721888162
0.008303142710914526
0.00421882224650025
0.0031473721697582156
0.0018647329003785133
-0.001114381117945888
0.0004759941169185902
0.0014445708543079515
0.00012575319522889895
-0.0021551595915348157
-0.0031603812693774304
-0.002347842026720367
-0.002749559788902716
-0.0013063120905661635
-0.0003618090503820212
-0.0014150244075791538
-0.00020036680013678544
-0.00190393761580808
-0.00358301634406977
-0.0027108396917335832
-0.0007857487545294735
-0.0021098443349054806
-0.0035701138420036315
-0.004088593386230451
-0.0029294706933825927
-0.0023383474389084427
-0.0014134386651483567
-0.00245225687376396
-0.003443561548860546
0.0009649213411683143
0.0002407093103332617
-0.003441280250474006
-0.0026485499307508263
-0.0024715639713984196
-0.00182563296290

-0.008698578219202778
-0.008417733074196704
-0.008021331076740513
-0.008062447426336012
-0.009096284622449574
-0.008707620356642484
-0.008447015422026796
-0.008282759453227
-0.008214518915844108
-0.008482378017278016
-0.008349036210815869
-0.00861370921370294
-0.008548933082085954
-0.008549894519334553
-0.008088370525328875
-0.008323580480012284
-0.008627702842957882
-0.008058444044479522
-0.008665855153278943
-0.008556968045697064
-0.008273856348587707
-0.008542881984596342
-0.008305036499724267
-0.008422386804955745
-0.009031294821128842
-0.008394008199313891


In [329]:
utils.eval(testloader, model, NLL, cuda=False, regression=True)

{'accuracy': None, 'loss': 0.00376473987512018}

SWAG

In [297]:
# model = RegressionNet.base(num_inputs=11)
swag_model = SWAG(RegressionNet.base, no_cov_mat=True, max_num_models=20, num_inputs=x.shape[1])

In [298]:
utils.adjust_learning_rate(optimizer, lr=1e-4)

0.0001

In [299]:
mus = []
sigma2s = []
for i in range(100):
    train_res = utils.train_epoch(trainloader, model, NLL, optimizer, cuda=False, regression=True)
    swag_model.collect_model(model)
#     swag_model.sample(0.0)
#     swag_res = utils.eval(testloader, swag_model, NLL, cuda=False, regression=True)
#     print(swag_res)
    pred, target = utils.predictions(testloader, model, cuda=False, regression=True)
    mu = pred[:, 0]
    log_sigma2 = pred[:, 1]
    sigma2 = np.exp(log_sigma2)
#     if train_res['loss'] < 1.04:
    mus.append(mu[:, None])
    sigma2s.append(sigma2[:, None])
    print(train_res['loss'], NLL_(torch.from_numpy(mu), torch.from_numpy(sigma2), torch.from_numpy(target)))
mus = np.hstack(mus)
sigma2s = np.hstack(sigma2s)

-0.005988671858265569 tensor(0.0079)
-0.0056200391281884575 tensor(0.0106)
-0.005599050364169271 tensor(0.0166)
-0.004909198422725989 tensor(0.0061)
-0.006231211493577626 tensor(0.0088)
-0.005106955797826492 tensor(0.0056)
-0.005896788065808117 tensor(0.0088)
-0.006152766916674613 tensor(0.0129)
-0.006276586418879487 tensor(0.0071)
-0.006328958900752016 tensor(0.0073)
-0.005810057572850006 tensor(0.0093)
-0.006152805696725984 tensor(0.0068)
-0.005721862116295789 tensor(0.0053)
-0.006079263822491915 tensor(0.0098)
-0.00650232204523458 tensor(0.0141)
-0.006288965755599659 tensor(0.0086)
-0.005728356835249893 tensor(0.0056)
-0.006234924758772252 tensor(0.0073)
-0.0058159057482265455 tensor(0.0071)
-0.005510589917519272 tensor(0.0064)
-0.006014433299300424 tensor(0.0063)
-0.006711146006350484 tensor(0.0127)
-0.005517102784014596 tensor(0.0075)
-0.00619590014561919 tensor(0.0052)
-0.006354206198946052 tensor(0.0056)
-0.006243775034460259 tensor(0.0061)
-0.0062030316672803025 tensor(0.0117)


In [300]:
print(np.linalg.norm(target - mus[:, 0]))
print(np.linalg.norm(target - np.mean(mus, axis=1)))

7.571209
7.563717


In [301]:
mu_ens = np.mean(mus, axis=1)
sigma2_ens = np.mean(sigma2s, axis=1) + (np.mean(mus**2, axis=1) - mu_ens**2)
NLL_(torch.from_numpy(mu_ens), torch.from_numpy(sigma2_ens), torch.from_numpy(target))

tensor(0.0063)

Ensembling

In [302]:
def upd_variances(model):
    variances = {}
    for module, name in model.params:
        mean = module.__getattr__('%s_mean' % name)
        sq_mean = module.__getattr__('%s_sq_mean' % name)
        var = sq_mean - mean**2
        var[var < 1e-6] = 1e-6
        sq_mean = var + mean**2
        module.__getattr__('%s_sq_mean' % name).copy_(sq_mean)
#         variances[name] = (mean, var)
#     return variances

In [307]:
upd_variances(swag_model)

In [308]:
mus = []
sigma2s = []
for i in range(30):
    swag_model.sample(.1)
    pred, target = utils.predictions(testloader, swag_model, cuda=False, regression=True)
    mu = pred[:, 0]
    log_sigma2 = pred[:, 1]
    sigma2 = np.exp(log_sigma2)
    mus.append(mu[:, None])
    sigma2s.append(sigma2[:, None])
mus = np.hstack(mus)
sigma2s = np.hstack(sigma2s)
# utils.eval(testloader, swag_model, NLL, cuda=False, regression=True)

In [309]:
mu_ens = np.mean(mus, axis=1)
sigma2_ens = np.mean(sigma2s, axis=1) + (np.mean(mus**2, axis=1) - mu_ens**2)

In [310]:
NLL_(torch.from_numpy(mu_ens), torch.from_numpy(sigma2_ens), torch.from_numpy(target))

tensor(0.0062)

Dropout

In [311]:
def train_dropout(m):
    if type(m)==torch.nn.modules.dropout.Dropout:
        m.train()

In [318]:
model = RegressionNet.base(num_inputs=x.shape[1], p=0.01)

In [319]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=1e-3,
    momentum=0.9,
    weight_decay=5e-2
)

In [320]:
for i in range(600):
    train_res = utils.train_epoch(trainloader, model, NLL, optimizer, cuda=False, regression=True)
    print(train_res['loss'])
    if i == 400:
        utils.adjust_learning_rate(optimizer, lr=1e-5)

0.5402362837603581
0.15388966759535067
0.12901668463236723
0.11344834771939595
0.09262173675799285
0.10322420473485552
0.0985243903846587
0.07901306988370427
0.09098628129420304
0.07119136466068779
0.08446133922387078
0.07745690151638941
0.07847753289497324
0.09107172715513012
0.11626355776580807
0.0816406598369802
0.07994911788028647
0.08548523686223157
0.08538187467640748
0.08400292147823554
0.0884906737601739
0.07932468096445197
0.07276114854313136
0.06950927805187962
0.08578159272408578
0.0788713868552973
0.06854897974751827
0.09094947403388327
0.09763830840836805
0.08604875016594553
0.08184230536648156
0.07644927836785873
0.07272750091994352
0.09222482717102266
0.08059740594584218
0.09518366356425384
0.08256989174978116
0.07624458828591647
0.08213331453366673
0.07657771573403394
0.07650914548747756
0.09012736622194369
0.07832227790415211
0.07294549218399186
0.07575962129119844
0.08023765555118631
0.07615211614678094
0.06782721731807002
0.07949292581022692
0.09077521350488695
0.072

0.06159819929237884
0.05819298049573242
0.05988691911757193
0.06978463978803942
0.0540695065835145
0.07026402157822313
0.04688631634691592
0.05521836794844461
0.07288472165789785
0.06378724114458233
0.04587315088357539
0.06250187709465521
0.06329998595140604
0.06756369685644943
0.06231305645823119
0.05152142679161096
0.07594056885969402
0.0657826301638734
0.061531113712661095
0.06425784732884694
0.05403401350403286
0.06183905511724175
0.062355864270889946
0.05952080459548211
0.0627036313623131
0.06442611854107673
0.0527094538626307
0.05886055375537506
0.06300258051918547
0.05677545466625224
0.0594008770945346
0.05560034309969091
0.06443465938967925
0.060893544875541335
0.06104174792427397
0.05977396664061816
0.05290725504655675
0.057350069937695196
0.05781542624871139
0.04857472861329516
0.07386178357781689
0.05512175062069313
0.061851439264479335
0.05450417392969049
0.06843971623982333
0.045800444632924375
0.05399316392173586
0.06712663811011232
0.05296586588545967
0.05622456193787062

In [321]:
model.eval()
train_dropout(model)

In [322]:
mus = []
sigma2s = []
for i in range(30):
    pred, target = utils.predictions(testloader, model, cuda=False, regression=True)
    mu = pred[:, 0]
    log_sigma2 = pred[:, 1]
    sigma2 = np.exp(log_sigma2)
    mus.append(mu[:, None])
    sigma2s.append(sigma2[:, None])
mus = np.hstack(mus)
sigma2s = np.hstack(sigma2s)
# utils.eval(testloader, swag_model, NLL, cuda=False, regression=True)

In [323]:
mu_ens = np.mean(mus, axis=1)
sigma2_ens = np.mean(sigma2s, axis=1) + (np.mean(mus**2, axis=1) - mu_ens**2)

In [324]:
NLL_(torch.from_numpy(mu_ens), torch.from_numpy(sigma2_ens), torch.from_numpy(target))

tensor(0.0223)